Extract TARs to DIR. Used on Cropscape data

In [ ]:
import tarfile
from pathlib import Path

target_dir = Path("")
targets = list(target_dir.glob("*"))
destination_dir = Path("")

In [ ]:
for target in targets:
  print(str(destination_dir / target.stem[0:-4]))
  tar = tarfile.open(str(target))
  tar.extractall(path=str(destination_dir / target.stem[0:-4]))
  tar.close()

Generate CARS

In [1]:
from pathlib import Path
import os
import subprocess
import pandas as pd
import json
year="2012"
TMPDIR=r""
target_dir = Path(f"") # path to input fils
car_dir = Path(f"") # path to output CARs
pow_path = r"" # path to POW executable (obtain from Textile POW Github)

In [148]:
# target_dir = Path(f"") # path to input fils
# car_dir = Path(f"") # path to output CARs


In [149]:
car_dir.mkdir(parents=True, exist_ok=True) # create car dir automatically

In [150]:
targets = [x for x in target_dir.glob("*") if x.is_dir()]

In [ ]:
targets

In [152]:
import datetime
print(datetime.datetime.now())

2022-09-28 17:30:50.405633


In [ ]:
df = pd.DataFrame(columns = ['name', 'payload_cid', 'piece_size', 'piece_cid', "file"])
for target in targets:

    print(target)

    car_target = car_dir / Path(str(target.stem) + ".car")

    if car_target.exists():
        print(f"Skipping {car_target.name}")
        continue

    # Don't aggregate 
    # result = subprocess.run([str(pow_path), "offline", "prepare", "--json", str(target), str(car_target)], shell=True, capture_output=True)
    
    # Aggregate
    result = subprocess.run(f"{str(pow_path)} offline prepare --json --aggregate {str(target)} {str(car_target)} --tmpdir {TMPDIR}", shell=True, capture_output=True)
    #limited tmp folder on GEOG cluster; add tmpdir to redirect tmp files
    
    try:
        result=json.loads(result.stderr.decode('utf-8'))
        temp_df = {"name": str(car_target.stem), "payload_cid": result["payload_cid"], "piece_size": result["piece_size"], "piece_cid": result["piece_cid"], "file": str(car_target.name)}
        df = df.append(temp_df, ignore_index=True)
        df.to_csv(f"{str(car_dir.parent)}/{str(car_dir.stem)}_car_master.csv") # CHANGE AS REQUIRED
        with open(f"{str(car_dir.parent)}/{str(car_dir.stem)}_{str(target.stem)}_car_reference.json",'w',encoding='utf-8') as fw:
            json.dump(result,fw,ensure_ascii=False, indent=4)
    except Exception as e:
        print(f"error - {target}  - {e}")
    
    print(result)

In [129]:
# with open(f"{str(car_dir.parent)}/{str(car_dir.stem)}_{str(target.stem)}_car_reference.json",'w',encoding='utf-8') as fw:
#     json.dump(result,fw,ensure_ascii=False, indent=4)

In [ ]:
result

In [106]:

print(datetime.datetime.now())

2022-08-31 00:35:18.041787


## upload GEOG Cluster

In [ ]:
#shell sync.sh

## upload S3

Used to upload files to S3. Now that we've switched to GEOG Cluster - this won't be used

In [ ]:
!pip install boto3

In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError
from pathlib import Path
import pandas as pd


ACCESS_KEY = ""
SECRET_KEY = ""

PATH_TO_CSV = ""
car_dir = Path("")

s3 = boto3.client('s3', aws_access_key_id=ACCESS_KEY,
                      aws_secret_access_key=SECRET_KEY)

df = pd.read_csv(PATH_TO_CSV)

def upload_to_aws(local_file, bucket, s3_file):
    
    try:
        s3.upload_file(local_file, bucket, s3_file)
        print("Upload Successful")
        return True
    except FileNotFoundError:
        print("The file was not found")
        return False
    except NoCredentialsError:
        print("Credentials not available")
        return False



for idx, row in df.iterrows():
  print("Index ", idx)
  print(row["name"])
  car_target = car_dir / str(row["file"])

  if not car_target.exists():
    print(row["name"])
    print("does not exists")
    continue

  print(str(car_target), '<NAME>', row["file"])
  uploaded = upload_to_aws(str(car_target), '<NAME>', row["file"])

  if not uploaded:
    print(row["name"])
    print("not uploaded")